In [ ]:
# Extract bits n-m of a virtual address x
def extract(x, n, m):
    return (x >> m) & (2**(n-m+1)-1)

# Three CRC hash functions H1, H2, and H3 that operate on pte_tag
def h1(x, len):
    return crc(x) % len

def h2(x, len):
    return crc(x ^ 0xFFFFFFFF)  % len

def h3(x, len):
    return crc(x ^ 0xAAAAAAAA)  % len

# CRC hash function for 33-bit integers
def crc(x):
    crc = 0xFFFFFFFF
    for i in range(33):
        if (((x >> i) & 1) ^ (crc >> 31)) == 1:
            crc = ((crc << 1) ^ 0x04C11DB7) & 0xFFFFFFFF
        else:
            crc = (crc << 1) & 0xFFFFFFFF
    return crc

# Takes the value performs the hashing and stores in a list 
def ilist_gen(x, len):
    i1 = h1(x, len)
    i2 = h2(x, len)  
    i3 = h3(x, len)
    return [i1, i2, i3]

In [ ]:
import numpy as np

TABLE_SIZE = 2**30                               # 1GB
PAGE_SIZE = 2**12                                # 4KB
BYTES_PER_LINE = 8                               # 8B
FRAME_LEN = PAGE_SIZE // BYTES_PER_LINE          # 512
NUM_FRAMES = 500                                 # 500 would do, ideally, TABLE_SIZE // PAGE_SIZE, 262144

# Initalising the PT table
pt_table = np.full((NUM_FRAMES, FRAME_LEN), None)

In [ ]:
import random

# d-ary hash table with len entries in each ary 
len = 150             # Length of each hash table
ary = 3               # Number of hash tables
max_attempts = 10     # Maximum number of attempts to insert a pte_tag        

# Intialising the hash table
pte_table = np.full((ary, len, 2), -1)

kernel_fp = [0]       # Kernel page-allocator frame pointer
count = [0]           # Number of page hits

# Inserting a pte_tag into the hash table
def insert1(x):

    dlist = [0, 1, 2]

    prev_d = -1
    pte_tag = extract(x, 47, 21)
    pte_offset = extract(x, 20, 12)
    pte_frame = kernel_fp[0]

    for j in range(1, max_attempts):
        ilist = [h1(pte_tag, len), h2(pte_tag, len), h3(pte_tag, len)]
        for d in dlist: 
            if pte_table[d][ilist[d]][0] == pte_tag:
                if pte_table[d][ilist[d]][1] == -1: 
                    pte_table[d][ilist[d]][1] = pte_frame
                    if j == 1: 
                        kernel_fp[0] += 1
                        if pt_table[pte_table[d][ilist[d]][1]][pte_offset] == None:
                            pt_table[pte_table[d][ilist[d]][1]][pte_offset] = 1
                        else:
                            count[0] += 1
                    return j, True 
                else:
                    if j == 1:
                        if pt_table[pte_table[d][ilist[d]][1]][pte_offset] == None:
                            pt_table[pte_table[d][ilist[d]][1]][pte_offset] = 1
                        else:
                            count[0] += 1
                    return j, True
            elif pte_table[d][ilist[d]][0] == -1:
                pte_table[d][ilist[d]][0] = pte_tag
                pte_table[d][ilist[d]][1] = pte_frame
                if j == 1: 
                    kernel_fp[0] += 1
                    if pt_table[pte_table[d][ilist[d]][1]][pte_offset] == None:
                        pt_table[pte_table[d][ilist[d]][1]][pte_offset] = 1
                    else:
                        count[0] += 1
                return j, True
            
        curr_d = random.choice(dlist)
  
        if(prev_d != -1): dlist.append(prev_d)

        update_pte_tag = pte_tag  
        update_pte_frame = pte_frame

        pte_tag = pte_table[curr_d][ilist[curr_d]][0]
        pte_frame = pte_table[curr_d][ilist[curr_d]][1]

        pte_table[curr_d][ilist[curr_d]][0] = update_pte_tag
        pte_table[curr_d][ilist[curr_d]][1] = update_pte_frame

        if j == 1: 
            kernel_fp[0] += 1
            if pt_table[pte_table[curr_d][ilist[curr_d]][1]][pte_offset] == None:
                pt_table[pte_table[curr_d][ilist[curr_d]][1]][pte_offset] = 1
            else:
                count[0] += 1

        prev_d = curr_d
        dlist.remove(curr_d)
        
    return j, False

In [ ]:
import time
 
# Number of insertion failures and successes
yep = 0
nah = 0

prev_j = 1

start = time.time()
with open("pinatrace_gups.out") as file:
    i = 0
    for line in file:
        i += 1
        if line != '#eof\n':
            virtual_address = line[18:32]
            virtual_address = int(virtual_address, 0)
            j, outcome = insert1(virtual_address)
            
            prev_j = (j + prev_j)/2

            if(outcome == False):
                nah += 1
            else:
                yep += 1

        if(i % 100000000 == 0):
            print(i, time.time() - start)
        
end =  time.time()

print("Execution time in seconds: ",(end-start))

In [ ]:
# Average attempts for insertion 
prev_j

In [ ]:
# Page hit rate
count

In [ ]:
# Number of PT frames allocated
kernel_fp[0]

In [ ]:
entry_null = 0 
entry_fill = 0 

for d in range(ary) :
    for i in range(len):
        if pte_table[d][i][0] == -1:
            entry_null += 1
        else: 
            entry_fill += 1

In [ ]:
# Number of filled entries in the hash table
entry_fill

In [ ]:
# Number of empty entries in the hash table
entry_null

In [ ]:
fill_rate = 0
empty_rate = 0

In [ ]:
for j in range(kernel_fp[0]):
    for i in range(FRAME_LEN):
        if pt_table[j][i] == None:
            empty_rate += 1
        else:
            fill_rate += 1

In [ ]:
# Number of filled entries in the PT table
fill_rate

In [ ]:
# Number of empty entries in the PT table
empty_rate

In [ ]:
# Insertion success rate
yep

In [ ]:
# Insertion failure rate 
nah